# Извлечение признаков из электромиограмм

In [1]:
import warnings
warnings.simplefilter('ignore')
from pyedflib import EdfReader
from matplotlib import pyplot as plt
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
import pandas as pd
import numpy as np

### Параметры:
Построение таблицы признаков (pd.DataFrame) происходит с шагом в одну секунду на скользящем временном окне.

In [2]:
WINDOW_LENGTH=20             #Длина скользящего окна в секундах(нечётное значение)
PATH='data/'                 #Путь к данным
CHANNELS_LIST='channels.xlsx'#Список каналов для извлечения признаков

### Описание признаков
Наследованый класс настройки признаков пакета tsfresh (https://github.com/blue-yonder/tsfresh)
Полный список доступных признаков(http://tsfresh.readthedocs.io/en/latest/text/list_of_features.html)

Предварительно отобраны:

abs-energy [Абсолютная энергия сигнала](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.abs_energy)
    
maximum [Максимум](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.maximum)
    
minimum [Минимум](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.minimum)
    
median [Медиана](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.median)
    
mean [Среднее](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.mean)
    
mean_abs_change [Среднее модуля производной](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.mean_abs_change)
    
mean_change [Среднее производной](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.mean_change)
    
standart_deviation [Стандартное отклонение](http://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.standard_deviation)
    

In [3]:
class EmgFCPParameters(MinimalFCParameters):
    """
    Класс настройки списка извлекаемых признаков
    """
    def __init__(self):
        MinimalFCParameters.__init__(self)
        
        to_del=['length','variance','sum_values']
        to_add=['abs_energy','mean_abs_change','mean_change']
        
        for param in to_del:
            del self[param]
        
        for param in to_add:
            self[param]=None
            

In [4]:
def movie_window(file,chnum,wsec=WINDOW_LENGTH):
    """
    Функция скользящего окна заданной длительности, на выходе получаем numpy.array,
    содержащий оконные срезы для каждой секунды
    """
    frq=file.getSampleFrequency(chnum)
    record=file.readSignal(chnum)
    rlen=file.getNSamples()[chnum]
    
    rsec=rlen//frq
    record_windows=np.zeros((rsec-(wsec//2)*2,wsec*frq))
    
    for i,row in enumerate(record_windows):
        record_windows[i]=record[i*frq:(i+wsec)*frq]
    
    return record_windows

In [5]:
def to_tsfresh_DataFrame(record_windows,wsec=WINDOW_LENGTH):
    """
    Функция формирующая DataFrame для извлечения признаков с помощью tsfresh,
    из numpy.array поллученного movie_window
    """
    df=pd.DataFrame(record_windows)
    df=df.stack(level=0).reset_index()
    df.columns=['wtime_sec','samples','values']
    df['wtime_sec']=df['wtime_sec']+WINDOW_LENGTH//2+1
    return df

In [6]:
def get_params(records,params=EmgFCPParameters()):
    """
    Функция извлечения признаков из DataFrame, полученного с помощью to_tsfresh_DataFrame
    """
    return extract_features(records,
                            default_fc_parameters=params,
                            column_id='wtime_sec',
                            column_sort="samples")

In [7]:
def movie_window_params(filename,chnum,path=PATH,wsec=WINDOW_LENGTH):
    """
    Извлечение признаков из электромиограммы на скользящем временном окне
    """
    EDFData=EdfReader(path+filename)
    record_windows=movie_window(EDFData,chnum,wsec=WINDOW_LENGTH)
    dataf=to_tsfresh_DataFrame(record_windows)
    params=get_params(dataf)
    params['File']=filename
    params['Channel']=chnum
    EDFData._close()
    return params

In [8]:
def multifiles_params(channels=CHANNELS_LIST,path=PATH):
    """
    Извлечение признаков из нескольких электромиограмм, объединение в одну таблицу и сохранение их в .csv файл
    """
    channels_df=pd.read_excel(path+channels)
    channels_df['Channel']=channels_df['Channel']-1
    params_df_list=[]
    
    for i,row in channels_df.iterrows():
        print('File '+str(row['File'])+', Channel '+str(row['Channel']))
        params_df_list.append(movie_window_params(str(row['File'])+'.edf',row['Channel']))
    
    params_df = pd.concat(params_df_list).reset_index()
    params_df.index.names=['Index']
    params_df.to_csv(PATH+'channels'+'_params.csv')
    
    return params_df

In [9]:
%%time
params_df_list=multifiles_params()

File 1, Channel 18


Feature Extraction: 100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


File 2, Channel 6


Feature Extraction: 100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


File 2, Channel 8


...

File 6, Channel 13


Feature Extraction: 100%|██████████| 10/10 [00:08<00:00,  1.12it/s]


File 7, Channel 20


Feature Extraction: 100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


CPU times: user 4min 40s, sys: 1min 43s, total: 6min 23s
Wall time: 7min 31s
